OPENAI API KEY 와 TAVILY API KEY 필요

In [19]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [36]:
from dotenv import load_dotenv

load_dotenv()

True

#### **상태 설정**

In [37]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages] # messages key값을 가지고, state에 대화가 나눠지는 모든 내용들을 쌓아나가도록 하기위해 Annotated를 가져와 list형태로 받아줄거고 add_messages를 통해서 list형태의 메세지를 계속 받아줌.

#### **ToolNode로 도구 노드 구축**

In [38]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2) # TavilySearchResults AI를 활용해서 최대 2개까지의 웹검색을 가져오는 API 도구를 선언
tools = [tool]
tool_node = ToolNode(tools)

# ToolNode가 무엇인지 아래 코드를 통해 알아보자.

#### **Tool Node가 무엇일까**
ToolNode란 사용자가 정의한 특정 Tool을 Node로 변환해주는 라이브러리

ToolNode는 어떻게 생겼길래 우리가 만든 어떤 Tool을 list형태로 보내주기만 하면 node로 변경이 가능할까?
아래 코드에서 확인해보자.

In [39]:
import json

from langchain_core.messages import ToolMessage

# ToolNode는 이런식으로 생겼다.(BasicToolNode) 두개의 함수를 포함하고있는 하나의 class다.
# langchain의 ToolMessage를 기반으로 해서 보여주고 있다.
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage.""" # 마지막 AIMessage에서 요청한 도구를 실행하는 노드

    def __init__(self, tools: list) -> None: # list를 ToolNode안에 넣어주면 
        self.tools_by_name = {tool.name: tool for tool in tools} # 이름을 받아주는 생성자(self.tools_by_name)를 선언하고. tools라는 list안에 있는 모든 도구들을 tool의 이름을 지정하도록 한다. (tool.name: tool)

    def __call__(self, inputs: dict): # call함수를 실행하면  
        if messages := inputs.get("messages", []): # 주어진 메세지를 선언하고,
            message = messages[-1] # 가장 첫번째 메세지를 가져온다.
        else:
            raise ValueError("No message found in input") # 여기서 에러가나면 input값에서 메세지를 찾을 수 없다고 value 에러를 뱉는다.
        outputs = []
        for tool_call in message.tool_calls: # 그리고 여기 message에 있는 tool_calls에 대해서 하나씩 tool_call을 실행 할건데  
            tool_result = self.tools_by_name[tool_call["name"]].invoke( # 위에 선언한 생성자 tools_by_name를 통해서 tool리스트 에서 각각의 이름 "name"에 대해서 invoke함수를 실행하고  
                tool_call["args"] # 실행할 사항으로써 tool_call에 있는 args를 실행한다. 이렇게하면 주어진 Tool 리스트에 대해서 각각의 도구의 이름을 지어주고 args를 통해서 ai로 부터 받은 도구사용 상세 사항을 직접 invoke함수에 넣어서 그 도구들을 실행할 수 있게 해준다.
            )
            outputs.append( # 위에 선언한 outputs[] 리스트에 
                ToolMessage( # ToolMessage들을 추가해준다.
                    content=json.dumps(tool_result), # tool_result를 json형태로 content에 담아주고
                    name=tool_call["name"], # 이름은 실행한 tool의 이름
                    tool_call_id=tool_call["id"], # id는 tool_call["id"]라고 선언
                )
            )
        return {"messages": outputs} # outputs 리스트를 messages 키 값에 할당해준다.

        # 이렇게 하면 BasicToolNode는 Tool 리스트를 받았을 때, 이것을 사용자로 부터 질문을 받으면 ai가 이것을 툴에게 보낼 tool calls라는 메세지 형태로 보내게 되고
        # 그것을 리스트안에있는 도구들이 순회하면서 각각 실행하게 되는 것이다.

### 정리
  이렇게 하면 BasicToolNode는 Tool 리스트를 받았을 때, 이것을 사용자로 부터 질문을 받으면 ai가 이것을 툴에게 보낼 tool calls라는 메세지 형태로 보내게 되고

  그것을 리스트안에있는 도구들이 순회하면서 각각 실행하게 되는 것이다.

  **즉, ToolNode는 우리가 tool을 정하기만 하면 이것을 실행가능한 함수로 변환해주는 것이다.**

  위에 ToolNode로 도구 노드를 구축하는 코드를 다시 살펴보자

  tool = TavilySearchResults(max_results=2) # TavilySearchResults AI를 활용해서 최대 2개까지의 웹검색을 가져오는 API 도구를 선언.

  tools = [tool] # 위 정의한 tool을 리스트에 담아 tools에 할당

  tool_node = ToolNode(tools) # ToolNode로 우리가 선언한 tool을 실행가능한 함수로 변환하여 tool_node에 할당

  #### Tool들은 직접 정의하지 않아도 langchain_community에서 도구 라이브러리들을 직접 가져다 쓸 수 있다.

#### **LLM 챗봇 설정**

In [40]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini") # LLM 모델을 설정
llm_with_tools = llm.bind_tools(tools) # 모델에게 이런 Tool들이 있다는걸 bind_tools로 통해서 알려줌

# 에이전트 노드 만들기 
def chatbot(state: State): # state값을 입력을 받고 
    result = llm_with_tools.invoke(state["messages"]) # state값에 있는 "messages" key값에 해당하는 것을 llm_with_tools에 invoke함수로 실행. (이렇게 하면 이 llm에게 state["messages"]에 있는 값을 넣어서 실행을 함)
    return {"messages": [result]} # 그것으로 나온 결과물을 "messages" key값에 직접 value로 넣어준다.

#### **그래프 구축**

In [41]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State) # 맨 위에 이미 정해준 State로 정해주고 graph_builder로 선언

# 두 가지 노드를 설정
graph_builder.add_node("chatbot", chatbot) # llm을 실행할 챗봇과
graph_builder.add_node("tools", tool_node) # llm 실행 결과중 요소가 있는 경우 실행할 ToolNode를 선언

graph_builder.add_edge("tools", "chatbot") # 엣지를 통해 chatbot과 tools사이에 엣지를 그어준다. (시작점은 tools, 끝점은 chatbot) tools가 실행된 경우에는 무조건 chatbot에 응답결과를 전달하도록 설정
graph_builder.add_conditional_edges("chatbot", tools_condition) # add_conditional_edges라고 해서 이 에이전트가 도구를 활용할 필요가 있는 경우에는 ToolNode도 엣지가 이어지도록 하고, 그렇지 않으면 end로 설정
                                                                # tools_condition는 랭그래프에서 만든 라이브러리. (이전에는 tool calls라는 속성이 llm에 응답에 포함된 경우에는 ToolNode로 가고 아니면 end로 가게 만들어줬지만 이걸 매번 실행하기가 번거롭기 때문에 라이브러리를 제공)
graph_builder.set_entry_point("chatbot") # set_entry_point로 챗봇을 설정해서 사용자의 질문이 들어오면 곧바로 챗봇 노드가 전달받을 수 있도록 만들어준다.
graph = graph_builder.compile() # 마지막으로 컴파일을 통해 실행가능한 graph로 만들어준다.

**인터넷 검색이 필요한 질문**

In [44]:
graph.invoke({"messages": {"role": "user", "content": "한국이 최근에 계엄령이 일어났다고 하는데 언제 일어났어?"}})

# 지금 질문은 llm이 실시간성 정보니까 웹검색을 활용해야겠다. 라고 판단을 하여 결과를 보면 additional_kwargs={'tool_calls'... 이런식으로 tool_calls을 통해 답변을 한 것을 볼 수 있다.
# 사용자 질문(HumanMessage) -> AI(llm)가 tool_calls 속성을 만들어 도구한테 주어짐(AIMessage) -> 도구 호출 결과로 답변이 쭉 나옴(ToolMessage) -> 마지막으로 도구로 통한 답변을 정리하여 AI(llm)이 답변을 정리하여 뱉어줌 (AIMessage)

{'messages': [HumanMessage(content='한국이 최근에 계엄령이 일어났다고 하는데 언제 일어났어?', additional_kwargs={}, response_metadata={}, id='81fc57af-5efb-4805-a1a1-dc9909f1c94e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tBgfzIA52zNtViT3Juws3NF4', 'function': {'arguments': '{"query":"한국 계엄령 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 99, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-efda032c-25e4-4fa7-949a-4d7190e15eba-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '한국 계엄령 2023'}, 'id': 'call_tBgfzIA52zNtViT3Juws3NF4', 't

**LLM이 답할 수 있는 질문**

In [49]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

# 실시간 성이 필요하지 않고 llm이 직접 답할 수 있는 질문이면 AIMessage에 tool_calls가 호출하지 않고 바로 그대로 content에 담겨 답변을 한것을 볼 수 있다.

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', id='15a7fd9f-5afb-4c7d-91be-b99a8fef88c2'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 기업으로, 소프트웨어, 하드웨어, 전자 제품 및 서비스 분야에서 활동하고 있습니다. 1975년 빌 게이츠와 폴 앨런에 의해 설립된 이 회사는 세계에서 가장 큰 소프트웨어 제작업체 중 하나로 알려져 있습니다.\n\n마이크로소프트의 가장 유명한 제품은 다음과 같습니다:\n\n1. **운영 체제**: Windows 시리즈는 개인용 컴퓨터와 서버에서 널리 사용되는 운영 체제입니다.\n2. **오피스 제품군**: Microsoft Office는 워드, 엑셀, 파워포인트 등 다양한 생산성 도구를 포함하고 있습니다.\n3. **클라우드 서비스**: Azure는 마이크로소프트의 클라우드 컴퓨팅 플랫폼으로, 기업들이 클라우드 기반의 서비스를 구축하고 운영할 수 있게 돕습니다.\n4. **게임**: Xbox 게임 콘솔과 Xbox Live 서비스는 마이크로소프트의 게임 분야에서 중요한 제품입니다.\n5. **소프트웨어 개발**: Visual Studio와 같은 개발 도구를 제공하여 소프트웨어 개발자를 지원합니다.\n\n마이크로소프트는 또한 인공지능(AI), 가상 현실(VR), 증강 현실(AR), 그리고 다양한 혁신 기술 분야에서도 활발히 연구 및 개발을 진행하고 있습니다. 이 회사는 전 세계적으로 수많은 직원과 고객을 보유하고 있으며, 기술 혁신과 기업 사회적 책임에 중점을 두고 운영되고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 334, 'prompt_tokens': 89, 'total_tokens': 423}, 'model_name': 'gpt-4o-mini-2024-07-1